
## Spark from zero


#Pyspark vs SQL
 Regla mental que debes repetir en clase
- SQL = declarativo
- PySpark = transformaciones encadenadas sobre dataFrame


In [0]:
%sql

SELECT * FROM samples.bakehouse.sales_customers
LIMIT 10

In [0]:

name = "Rodrigo"
print(f"Mi nombre es {name}")

In [0]:
df = spark.read.table("samples.bakehouse.sales_customers")
df_selected = df.select("*")

df_limited = df.select("*").limit(4)
df_limited.display()


## DISTINCT

In [0]:
%sql

select distinct country
from samples.bakehouse.sales_customers


In [0]:
df_distinct = df_selected.select("country").distinct()

display(df_distinct)

#Where/ Filtro

In [0]:
%sql

Select *
from samples.bakehouse.sales_customers
where 1=1
and country = 'Japan'
and gender = 'female'
limit 5
;

In [0]:
from pyspark.sql.functions import col

df_filtered = df_selected.filter(
    (col("country") == "USA") & (col("gender") == "female")
)
display(df_filtered)


#ORDER BY + LIMIT

In [0]:
df_sorted = df.orderBy(col("first_name").desc())

display(df_sorted.limit(4))

In [0]:
%sql
select 
  concat(first_name,"",last_name) as full_name,
  concat_ws(",",city,state,country,continent) as address,
  35 as age,
  "random comment" as comment,
  true as is_true
from samples.bakehouse.sales_customers
Limit 10

In [0]:
from pyspark.sql.functions import col, concat, lit, concat_ws

df_plus_columns = df.select(
    concat(col("first_name"), lit(""), col("last_name")).alias("full_name"),
    concat_ws(",", col("city"), col("state"), col("country"), col("continent")).alias("address"),
    lit(35).alias("age"),
    lit("random comment").alias("comment"),
    lit(True).alias("is_true")
)
display(df_plus_columns.limit(10))

#GROUP BY + AGGREGATION


In [0]:
%sql

select country , count(1) as cantidad
from samples.bakehouse.sales_customers
group by country
order by cantidad desc

In [0]:
from pyspark.sql.functions import count, col

df_grouped = (
    df.groupBy("country")
      .agg(count("*").alias("count"))
      .orderBy(col("count").desc())
)

display(df_grouped)


# JOIN (clave para silver)

In [0]:
%sql

select
  concat(c.first_name,' ',c.last_name) as full_name,
  c.country,
  c.gender,
  s.product,
  s.quantity,
  s.paymentMethod,
  s.dateTime

from samples.bakehouse.sales_transactions s
inner join samples.bakehouse.sales_customers c
on s.customerID= c.customerID
limit 10
 

In [0]:
df_sales = spark.read.table("samples.bakehouse.sales_transactions")
df_joined = (df.join(df_sales, on="customerID", how="inner")
             .select(df["*"])
            )
display(df_joined.limit(10))

In [0]:
%sql

SELECT
  CAST(transactionID AS STRING) AS transactionID,
  CAST(customerID AS STRING) AS customerID,
  CAST(franchiseID AS STRING) AS franchiseID,
  CAST(dateTime AS STRING) AS dateTime,
  CAST(product AS STRING) AS product
FROM samples.bakehouse.sales_transactions;

In [0]:
from pyspark.sql.functions import col

df_casted = df_sales.select(
    col("transactionID").cast("long").alias("transactionID"),
    col("customerID").cast("long").alias("customerID"),
    col("franchiseID").cast("long").alias("franchiseID"),
    col("dateTime").cast("timestamp").alias("dateTime"),
    col("product"),
    col("quantity").cast("long").alias("quantity"),
    col("unitPrice").cast("long").alias("unitPrice"),
    col("totalPrice").cast("long").alias("totalPrice"),
    col("paymentMethod"),
    col("cardNumber").cast("long").alias("cardNumber"),
    (col("unitPrice")*col("quantity")).alias("total") #multiplicacion de columnas
)
display(df_casted.limit(10))

In [0]:
from pyspark.sql.functions import lit

df_test = df_casted.withColumn("nuevacolumna", lit("nuevacolumna_literal"))

df_now = df_test.drop("nuevacolumna")

display(df_now)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS nombre_catalogo